In [ ]:
import pandas as pd
from tkinter import Tk, filedialog
import ast
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tabulate import tabulate
import pprint 
import json
import scipy.stats as stats

## Upload Database and Create Dataframe

In [ ]:
# Create Tk root
root = Tk()

# Hide the main window
root.withdraw()

# Set the main window to be always on top
root.call('wm', 'attributes', '.', '-topmost', True)

# Ask user to select CSV files
csv_paths = filedialog.askopenfilename(multiple=True, title='Select CSV files', filetypes=[('CSV files', '*.csv')])

# Initialize a list to store DataFrames for each selected CSV file
data_frames = []

# Iterate over each selected CSV file path
for csv_path in csv_paths:
    df = pd.read_csv(csv_path, sep=';')
    data_frames.append(df)

# Concatenate all DataFrames into a single DataFrame
original_df = pd.concat(data_frames, ignore_index=True)
original_df.update(combined_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x))


# Now combined_df contains data from all selected CSV files
#print(original_df)